In [24]:
"""
CHEMOSTAT Results Visualization Tool

Generates interactive HTML plots from CHEMOSTAT simulation results using Plotly.
Works for any number of species (n).

Organizes results by:
- One criteria (diversity/production): IC -> T -> plots
- Both criteria: IC -> T -> (α, β) -> plots

Expected structure:
results/
├── diversity/
│   ├── IC1-T1.csv, IC1-T2.csv, ...
├── production/
│   ├── IC1-T1.csv, IC1-T2.csv, ...
└── both/
    ├── IC1-T1-a1-b1.csv, IC1-T1-a2-b2.csv, ...

Usage:
    python plot_results.py
"""

import os
import re
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from pathlib import Path
from collections import defaultdict


def load_simulation_data(filepath):
    """Load simulation data from CSV file."""
    df = pd.read_csv(filepath)
    
    x_cols = [col for col in df.columns if col.startswith('x')]
    n_species = len(x_cols)
    
    return {
        't': df['t'].values,
        'x': df[x_cols].values.T,
        's': df['s'].values,
        'u': df['u'].values,
        'P': df['P'].values,
        'S': df['S'].values,
        'n_species': n_species
    }


def load_metadata(filepath):
    """Load simulation metadata from text file."""
    metadata = {}
    
    if not os.path.exists(filepath):
        return metadata
    
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if ':' in line and not line.startswith(' '):
                key, value = line.split(':', 1)
                metadata[key.strip()] = value.strip()
    
    return metadata


def parse_filename(filename, category):
    """
    Parse filename according to category.
    
    Returns dict with: ic, time, alpha, beta (if applicable)
    """
    stem = Path(filename).stem
    
    if category in ['diversity', 'production']:
        # Format: ICX-TY
        match = re.match(r'(IC\d+)-T(\d+)', stem)
        if match:
            return {
                'ic': match.group(1),
                'time': match.group(2),
                'alpha': None,
                'beta': None
            }
    else:  # both
        # Format: ICX-TY-aZ-bW
        match = re.match(r'(IC\d+)-T(\d+)-a([^-]+)-b(.+)', stem)
        if match:
            return {
                'ic': match.group(1),
                'time': match.group(2),
                'alpha': match.group(3),
                'beta': match.group(4)
            }
    
    return None


def create_comparison_plot(data_list, titles, main_title, subtitle=""):
    """
    Create side-by-side comparison plot for multiple simulations.
    
    Parameters
    ----------
    data_list : list of dict
        List of simulation data dictionaries
    titles : list of str
        Titles for each subplot
    main_title : str
        Main title for the entire figure
    subtitle : str
        Subtitle with additional info
        
    Returns
    -------
    plotly.graph_objects.Figure
    """
    n_sims = len(data_list)
    
    # Create subplots: 3 rows × n_sims columns
    fig = make_subplots(
        rows=3, cols=n_sims,
        subplot_titles=titles,
        vertical_spacing=0.10,
        horizontal_spacing=0.08,
        row_heights=[0.4, 0.3, 0.3],
        specs=[[{'type': 'scatter'} for _ in range(n_sims)] for _ in range(3)]
    )
    
    # Plot each simulation
    for col_idx, (data, title) in enumerate(zip(data_list, titles), start=1):
        n_species = data['n_species']
        show_legend = (col_idx == 1)  # Only show legend for first column
        
        # TOP PANEL: State variables
        for i in range(n_species):
            fig.add_trace(
                go.Scatter(
                    x=data['t'],
                    y=data['x'][i, :],
                    mode='lines',
                    name=f'x<sub>{i+1}</sub>',
                    line=dict(width=2, color=px.colors.qualitative.Plotly[i]),
                    legendgroup='states',
                    showlegend=show_legend
                ),
                row=1, col=col_idx
            )
        
        # Substrate
        fig.add_trace(
            go.Scatter(
                x=data['t'],
                y=data['s'],
                mode='lines',
                name='s',
                line=dict(width=2, color=px.colors.qualitative.Plotly[n_species]),
                legendgroup='states',
                showlegend=show_legend
            ),
            row=1, col=col_idx
        )
        
        # MIDDLE PANEL: Control
        fig.add_trace(
            go.Scatter(
                x=data['t'],
                y=data['u'],
                mode='lines',
                name='u(t)',
                line=dict(width=2.5, color='red'),
                legendgroup='control',
                showlegend=show_legend
            ),
            row=2, col=col_idx
        )
        
        # BOTTOM PANEL: Production and Biodiversity
        fig.add_trace(
            go.Scatter(
                x=data['t'],
                y=data['P'],
                mode='lines',
                name='P(x, u)',
                line=dict(width=2, color='green'),
                legendgroup='objectives',
                showlegend=show_legend
            ),
            row=3, col=col_idx
        )
        
        fig.add_trace(
            go.Scatter(
                x=data['t'],
                y=data['S'],
                mode='lines',
                name='S(x)',
                line=dict(width=2, color='orange'),
                legendgroup='objectives',
                showlegend=show_legend
            ),
            row=3, col=col_idx
        )
    
    # Update all axes
    for row in [1, 2, 3]:
        for col in range(1, n_sims + 1):
            fig.update_xaxes(
                title_text="t" if row == 3 else "",
                showgrid=True,
                gridwidth=1,
                gridcolor='rgba(128, 128, 128, 0.2)',
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=True,
                row=row, col=col
            )
            
            fig.update_yaxes(
                showgrid=True,
                gridwidth=1,
                gridcolor='rgba(128, 128, 128, 0.2)',
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=True,
                row=row, col=col
            )
    
    # Layout
    title_html = f"<b>{main_title}</b>"
    if subtitle:
        title_html += f"<br><sub>{subtitle}</sub>"
    
    fig.update_layout(
        title=dict(
            text=title_html,
            x=0.5,
            xanchor='center',
            font=dict(size=18)
        ),
        height=900,
        showlegend=True,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.98,
            xanchor="right",
            x=1.1,
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="rgba(0, 0, 0, 0.2)",
            borderwidth=1
        ),
        hovermode='closest',
        plot_bgcolor='white',
        font=dict(size=12)
    )
    
    return fig


def organize_simulations(results_dir):
    """
    Organize CSV files by category, IC, and time.
    
    Returns
    -------
    dict
        Structure: {category: {ic: {time: [files]}}}
    """
    results_dir = Path(results_dir)
    organized = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    
    # Look for subdirectories
    for category_dir in results_dir.iterdir():
        if not category_dir.is_dir():
            continue
        
        category = category_dir.name
        
        # Find all CSV files in this category
        csv_files = [f for f in category_dir.glob("*.csv") if '_metadata' not in f.name]
        
        for csv_file in csv_files:
            parsed = parse_filename(csv_file.name, category)
            
            if parsed:
                ic = parsed['ic']
                time = parsed['time']
                
                file_info = {
                    'file': csv_file,
                    'ic': ic,
                    'time': time,
                    'alpha': parsed['alpha'],
                    'beta': parsed['beta'],
                    'name': csv_file.stem
                }
                
                organized[category][ic][time].append(file_info)
    
    # Sort files within each time group
    for category in organized:
        for ic in organized[category]:
            for time in organized[category][ic]:
                files = organized[category][ic][time]
                if category == 'both':
                    # Sort by alpha, then beta
                    files.sort(key=lambda x: (float(x['alpha']), float(x['beta'])))
                else:
                    # Sort by filename
                    files.sort(key=lambda x: x['name'])
    
    return organized


def generate_all_plots(results_dir, output_dir="plots", open_in_browser=True):
    """
    Generate organized HTML plots for all simulations.
    """
    results_dir = Path(results_dir)
    output_dir = Path(output_dir)
    
    # Create output directory
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Organize simulations
    organized = organize_simulations(results_dir)
    
    if not organized:
        print(f"No simulations found in {results_dir}")
        return
    
    print(f"\n{'='*70}")
    print(f"CHEMOSTAT VISUALIZATION")
    print(f"{'='*70}")
    print(f"Output directory: {output_dir}")
    print(f"{'='*70}\n")
    
    # Structure for index: 
    # For diversity/production: {category: {ic: {'file': ..., 'n_times': ...}}}
    # For both: {category: {ic: [{'time': ..., 'file': ..., 'n_variants': ...}]}}
    index_structure = defaultdict(lambda: defaultdict(lambda: [] if 'both' else {}))

    x0_list = [ "[10.0, 0.1, 0.1, 0.1, 0.1]",
                "[10.0, 0.1, 10.0, 0.1, 0.1]",
                "[10.0, 0.1, 10.0, 0.1, 10.0]" ]

    # Process each category
    for category in sorted(organized.keys()):
        print(f"\nCategory: {category}")
        print(f"-" * 70)
        
        ics = organized[category]
        
        ic_index = -1
        for ic in sorted(ics.keys()):
            ic_index += 1
            print(f"  {ic}:")
            
            times = ics[ic]
            
            if category in ['diversity', 'production']:
                # For diversity/production: all times side-by-side in ONE plot per IC
                try:
                    data_list = []
                    titles = []
                    
                    # Collect all times for this IC
                    for time in sorted(times.keys(), key=lambda x: int(x)):
                        files = times[time]
                        # Should only be one file per time for diversity/production
                        if files:
                            file_info = files[0]
                            data = load_simulation_data(file_info['file'])
                            data_list.append(data)
                            titles.append(f"T={time}")
                    
                    print(f"    {len(data_list)} time points")
                    
                    # Create main title
                    main_title = f"Maximizing {category} for x<sub>0</sub>={x0_list[ic_index]},"
                    subtitle = " "
                    
                    # Create comparison plot with all times
                    fig = create_comparison_plot(data_list, titles, main_title, subtitle)
                    
                    # Save HTML (one file per IC)
                    output_file = output_dir / f"{category}_{ic}.html"
                    fig.write_html(
                        str(output_file),
                        config={
                            'displayModeBar': True,
                            'displaylogo': False,
                            'modeBarButtonsToRemove': ['pan2d', 'lasso2d']
                        }
                    )
                    
                    # Add to index structure (single entry per IC)
                    index_structure[category][ic] = {
                        'file': output_file.name,
                        'n_times': len(data_list)
                    }
                    
                    print(f"    ✓ Saved: {output_file}")
                    
                except Exception as e:
                    print(f"    ✗ Error: {e}")
                    import traceback
                    traceback.print_exc()
            
            else:  # category == 'both'
                # For both: separate plot for each time (varying α, β)
                for time in sorted(times.keys(), key=lambda x: int(x)):
                    files = times[time]
                    print(f"    T{time}: {len(files)} variant(s)")
                    
                    try:
                        # Load all data for this IC-Time combination
                        data_list = []
                        titles = []
                        
                        for file_info in files:
                            data = load_simulation_data(file_info['file'])
                            data_list.append(data)
                            title = f"α=0.{file_info['alpha']}, β=0.{file_info['beta']}"
                            titles.append(title)
                        
                        # Create main title
                        main_title = f"Max ∫αP+βS  for x<sub>0</sub>={x0_list[ic_index]}, T={time}"
                        subtitle = " "
                        
                        # Create comparison plot
                        fig = create_comparison_plot(data_list, titles, main_title, subtitle)
                        
                        # Save HTML
                        output_file = output_dir / f"{category}_{ic}_T{time}.html"
                        fig.write_html(
                            str(output_file),
                            config={
                                'displayModeBar': True,
                                'displaylogo': False,
                                'modeBarButtonsToRemove': ['pan2d', 'lasso2d']
                            }
                        )
                        
                        # Add to index structure
                        index_structure[category][ic].append({
                            'time': time,
                            'file': output_file.name,
                            'n_variants': len(files)
                        })
                        
                        print(f"      ✓ Saved: {output_file}")
                        
                    except Exception as e:
                        print(f"      ✗ Error: {e}")
                        import traceback
                        traceback.print_exc()
    
    # Create index page
    if index_structure:
        index_file = output_dir / "index.html"
        create_hierarchical_index(index_structure, index_file)
        print(f"\n✓ Created index page: {index_file}")
        
        total_plots = sum(
            len(times) 
            for cat in index_structure.values() 
            for times in cat.values()
        )
        
        print(f"\n{'='*70}")
        print(f"Generated {total_plots} plot(s)")
        print(f"Open {index_file} in your browser to view all results")
        print(f"{'='*70}\n")
        
        # Open in browser
        if open_in_browser:
            import webbrowser
            webbrowser.open(f'file://{index_file.absolute()}')


def create_hierarchical_index(index_structure, output_file):
    """
    Create hierarchical index HTML page.
    
    Structure:
    - One Criteria
      - diversity
        - IC1 (link - all times side-by-side)
        - IC2 (link - all times side-by-side)
      - production
        - IC1 (link - all times side-by-side)
        ...
    - Both Criteria
      - IC1
        - T1 (link - varying α, β)
        - T2 (link - varying α, β)
      ...
    """
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>CHEMOSTAT Simulation Results</title>
        <style>
            body {
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                max-width: 1400px;
                margin: 50px auto;
                padding: 20px;
                background-color: #f5f5f5;
            }
            h1 {
                color: #333;
                border-bottom: 3px solid #2ca02c;
                padding-bottom: 10px;
            }
            .main-section {
                background-color: white;
                border-radius: 8px;
                padding: 25px;
                margin: 25px 0;
                box-shadow: 0 2px 8px rgba(0,0,0,0.1);
            }
            .section-title {
                font-size: 26px;
                font-weight: bold;
                color: #d62728;
                margin-bottom: 20px;
                padding-bottom: 10px;
                border-bottom: 2px solid #e0e0e0;
            }
            .category {
                margin: 20px 0;
                padding: 15px;
                background-color: #f9f9f9;
                border-left: 4px solid #1f77b4;
                border-radius: 4px;
            }
            .category-title {
                font-size: 20px;
                font-weight: bold;
                color: #1f77b4;
                margin-bottom: 12px;
            }
            .ic-group {
                margin: 15px 0 15px 25px;
                padding: 12px;
                background-color: white;
                border-left: 3px solid #2ca02c;
                border-radius: 4px;
            }
            .ic-title {
                font-size: 18px;
                font-weight: 600;
                color: #2ca02c;
                margin-bottom: 10px;
            }
            .ic-item {
                padding: 8px 12px;
                margin: 6px 0 6px 25px;
                background-color: #f5f5f5;
                border-left: 3px solid #2ca02c;
                border-radius: 3px;
                transition: all 0.3s ease;
            }
            .ic-item:hover {
                background-color: #e8f4f8;
                transform: translateX(5px);
                box-shadow: 2px 2px 6px rgba(0,0,0,0.1);
            }
            .ic-item a {
                color: #2ca02c;
                text-decoration: none;
                font-size: 15px;
                font-weight: 500;
            }
            .ic-item a:hover {
                color: #1f7a1f;
                text-decoration: underline;
            }
            .time-list {
                margin-left: 25px;
            }
            .time-item {
                padding: 8px 12px;
                margin: 6px 0;
                background-color: #f5f5f5;
                border-left: 3px solid #ff7f0e;
                border-radius: 3px;
                transition: all 0.3s ease;
            }
            .time-item:hover {
                background-color: #e8f4f8;
                transform: translateX(5px);
                box-shadow: 2px 2px 6px rgba(0,0,0,0.1);
            }
            .time-item a {
                color: #ff7f0e;
                text-decoration: none;
                font-size: 15px;
                font-weight: 500;
            }
            .time-item a:hover {
                color: #d65c00;
                text-decoration: underline;
            }
            .variant-count {
                color: #666;
                font-size: 13px;
                margin-left: 8px;
                font-weight: normal;
            }
            .summary {
                background-color: #e8f4f8;
                padding: 15px;
                border-radius: 8px;
                margin-bottom: 20px;
            }
            .footer {
                margin-top: 40px;
                text-align: center;
                color: #666;
                font-size: 14px;
            }
        </style>
    </head>
    <body>
        <h1>🧬 CHEMOSTAT Simulation Results</h1>
        
        <div class="summary">
            <strong>Structure:</strong><br>
            • One Criteria: Category → IC (all times side-by-side)<br>
            • Both Criteria: IC → Time → (α, β) variations<br>
            • IC1 = [10,0,0,0,0], IC2 = [10,0,10,0,0], IC3 = [10,0,10,0,10]<br>
        </div>
    """
    
    # Separate into one criteria and both criteria
    one_criteria = {k: v for k, v in index_structure.items() if k in ['diversity', 'production']}
    both_criteria = {k: v for k, v in index_structure.items() if k == 'both'}
    
    # ONE CRITERIA SECTION
    if one_criteria:
        html_content += """
        <div class="main-section">
            <div class="section-title">📊 One Criteria</div>
        """
        
        for category in ['diversity', 'production']:
            if category not in one_criteria:
                continue
                
            ics = one_criteria[category]
            
            html_content += f"""
            <div class="category">
                <div class="category-title">→ Only {category}</div>
            """
            
            for ic in sorted(ics.keys()):
                ic_info = ics[ic]
                
                html_content += f"""
                <div class="ic-item">
                    <a href="{ic_info['file']}" target="_blank">
                        → {ic}
                    </a>
                    <span class="variant-count">(all {ic_info['n_times']} time{'s' if ic_info['n_times'] > 1 else ''} side-by-side)</span>
                </div>
                """
            
            html_content += """
            </div>
            """
        
        html_content += """
        </div>
        """
    
    # BOTH CRITERIA SECTION
    if both_criteria:
        html_content += """
        <div class="main-section">
            <div class="section-title">📊 Both Criteria</div>
        """
        
        ics = both_criteria['both']
        
        for ic in sorted(ics.keys()):
            times = ics[ic]
            
            html_content += f"""
            <div class="ic-group">
                <div class="ic-title">→ {ic}</div>
                <div class="time-list">
            """
            
            for time_info in sorted(times, key=lambda x: int(x['time'])):
                html_content += f"""
                    <div class="time-item">
                        <a href="{time_info['file']}" target="_blank">
                            → T{time_info['time']}
                        </a>
                        <span class="variant-count">(varying α, β: {time_info['n_variants']} plot{'s' if time_info['n_variants'] > 1 else ''})</span>
                    </div>
                """
            
            html_content += """
                </div>
            </div>
            """
        
        html_content += """
        </div>
        """
    
    html_content += """
        <div class="footer">
            <p>Generated with Plotly | Interactive plots - hover, zoom, and pan</p>
        </div>
    </body>
    </html>
    """
    
    with open(output_file, 'w') as f:
        f.write(html_content)


# =============================================================================
# MAIN EXECUTION
# =============================================================================

# if __name__ == "__main__":
#     import sys
    
#     if len(sys.argv) > 1:
#         results_dir = sys.argv[1]
#     else:
#         results_dir = "results"
    
#     if len(sys.argv) > 2:
#         output_dir = sys.argv[2]
#     else:
#         output_dir = "plots"
    
#     generate_all_plots(results_dir, output_dir, open_in_browser=True)

    
results_dir = "results"
output_dir = "plots"

generate_all_plots(results_dir, output_dir, open_in_browser=True)


CHEMOSTAT VISUALIZATION
Output directory: plots


Category: both
----------------------------------------------------------------------
  IC1:
    T10: 4 variant(s)
      ✓ Saved: plots/both_IC1_T10.html
    T20: 4 variant(s)
      ✓ Saved: plots/both_IC1_T20.html
    T30: 4 variant(s)
      ✓ Saved: plots/both_IC1_T30.html
    T40: 4 variant(s)
      ✓ Saved: plots/both_IC1_T40.html
  IC2:
    T10: 4 variant(s)
      ✓ Saved: plots/both_IC2_T10.html
    T20: 4 variant(s)
      ✓ Saved: plots/both_IC2_T20.html
    T30: 4 variant(s)
      ✓ Saved: plots/both_IC2_T30.html
    T40: 4 variant(s)
      ✓ Saved: plots/both_IC2_T40.html
  IC3:
    T10: 4 variant(s)
      ✓ Saved: plots/both_IC3_T10.html
    T20: 4 variant(s)
      ✓ Saved: plots/both_IC3_T20.html
    T30: 4 variant(s)
      ✓ Saved: plots/both_IC3_T30.html
    T40: 4 variant(s)
      ✓ Saved: plots/both_IC3_T40.html

Category: diversity
----------------------------------------------------------------------
  IC1:
    4 time 